# I - Introduction

# II - Prepare required frameworks

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as GBC
import lightgbm as lgbm
from sklearn.feature_selection import mutual_info_classif
from scipy import stats
import joblib

In [48]:
import warnings

# Ẩn tất cả các cảnh báo
warnings.filterwarnings('ignore')

In [49]:
# from sklearnex import patch_sklearn

In [50]:
# patch_sklearn()

# III - Prepare dataset

# 1. Load dataset

In [51]:
dataset = pd.read_excel('./Dry_Bean_Dataset.xlsx')

In [52]:
# dataset.info()

In [53]:
# dataset.duplicated().sum()

In [54]:
dataset = dataset.set_index('Bean ID')

In [55]:
dataset.columns

Index(['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
       'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2',
       'ShapeFactor3', 'ShapeFactor4', 'Class'],
      dtype='object')

In [56]:
features = dataset.columns[:-1]
target = dataset.columns[-1]

In [57]:
# x.describe()

In [58]:
x = dataset[features]
y = dataset[target]

In [59]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [60]:
# dataset['Class'] = pd.DataFrame(encoder.fit_transform(dataset['Class']), columns=['Class'])

In [61]:
pd.DataFrame(encoder.fit_transform(dataset['Class']), columns=['Class']).value_counts()

Class
3        3546
6        2636
5        2027
4        1928
2        1630
0        1322
1         522
Name: count, dtype: int64

In [62]:
# dataset['Class'].value_counts()

In [63]:
def prepare_data(dataset, features, target, scaler, encoder):
    x = dataset[features]
    y = dataset[target]
    y_encoded = encoder.fit_transform(y)
    x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=20)
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    return x_train, x_test, y_train, y_test

In [64]:
x_train, x_test, y_train, y_test = prepare_data(dataset, features, target, scaler, encoder)

# Training models

In [65]:
from hyperopt import hp
from hyperopt import fmin, tpe, Trials

# x_train, x_test, y_train, y_test = prepare_data(dataset, features, target, scaler, encoder)

# # Không gian tìm kiếm cho SVM
# space = {
#     'C': hp.loguniform('C', -4, 2),
#     'gamma': hp.loguniform('gamma', -4, -2),
#     'kernel': hp.choice('kernel', ['rbf', 'linear'])
# }

# # Định nghĩa hàm mục tiêu
# def objective(params):
#     model = SVC(**params)
#     score = cross_val_score(model, x_train, y_train, cv=5, n_jobs=-1)
#     accuracy = score.mean()
#     return -accuracy

# # Khởi tạo Trials để lưu trữ kết quả
# trials = Trials()

# # Thực hiện tìm kiếm hyperparameters
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

# # Chuyển đổi kết quả tốt nhất
# best['kernel'] = ['rbf', 'linear'][best['kernel']]

# # In ra kết quả tốt nhất
# print(f"Best parameters: {best}")

# # Đánh giá mô hình trên tập kiểm tra
# best_model = SVC(**best)
# best_model.fit(x_train, y_train)
# test_score = best_model.score(x_test, y_test)
# print(f"Test set score: {test_score}")


In [66]:
def hyperopt_search(space, model, x_train, y_train, x_test, y_test, algo=tpe.suggest, max_evals=100, categorical_params={}, trials=None, integer_params=[]):
    def objective(params, model=model, x_train=x_train, y_train=y_train, integer_params=integer_params):
        for key in integer_params:
            if key in params:
                params[key] = int(params[key])
        model = model(**params)
        score = cross_val_score(model, x_train, y_train, cv=5, n_jobs=-1)
        accuracy = score.mean()
        return -accuracy
    
    if trials is None:
        trials = Trials()
    
    best = fmin(fn=objective, space=space, algo=algo, max_evals=max_evals, trials=trials)
    
    for param, idx in categorical_params.items():
        best[param] = idx[best[param]]
        
    for key in integer_params:
        if key in best:
            best[key] = best[key].astype(int)
    
    best_model = model(**best)
    best_model.fit(x_train, y_train)
    test_score = best_model.score(x_test, y_test)
    print(f"Test set score: {test_score}")
    return best_model, best


In [67]:
### SVC

space = {
    'C': hp.loguniform('C', -2, 4),
    'gamma': hp.loguniform('gamma', -4, -2),
    'kernel': hp.choice('kernel', ['rbf', 'linear'])
}

category_params = {
    'kernel': ['rbf', 'linear']
}

best_SVC_model, best_SVC_params = hyperopt_search(space, model=SVC, algo=tpe.suggest, max_evals=100, categorical_params=category_params, 
                                                  x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

100%|██████████| 100/100 [01:30<00:00,  1.11trial/s, best loss: -0.9312094511743737]
Test set score: 0.9298567756151304


In [68]:
# print(f"Best parameters: {best_SVC_model.get_params()}")
print(f"Best parameters: {best_SVC_params}")

Best parameters: {'C': 2.182244301323868, 'gamma': 0.1217270894043437, 'kernel': 'rbf'}


In [84]:
# joblib.dump(best_SVC_model, 'best_SVC_model.pkl')

['best_SVC_model.pkl']

In [69]:
### Random Forest
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 150, 1),
    'max_depth': hp.quniform('max_depth', 5, 20, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1)
}

category_params = {}

int_params = ['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf']

# best_RF_model, best_RF_params = hyperopt_search(space, model=RFC, algo=tpe.suggest, max_evals=100, categorical_params=category_params, integer_params=int_params, 
                                                #   x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

In [83]:
# joblib.dump(best_RF_model, 'best_RF_model.pkl')

NameError: name 'best_RF_model' is not defined

In [70]:
# print(f"Best parameters: {best_RF_params}")

In [71]:
### KNN
space = {
    'n_neighbors': hp.quniform('n_neighbors', 1, 20, 1),
    'weights': hp.choice('weights', ['uniform', 'distance']),
    'p': hp.choice('p', [1, 2])
}

category_params = {
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

int_params = ['n_neighbors']

best_KNN_model, best_KNN_params = hyperopt_search(space, model=KNC, algo=tpe.suggest, max_evals=100, categorical_params=category_params, integer_params=int_params, 
                                                  x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

100%|██████████| 100/100 [00:27<00:00,  3.61trial/s, best loss: -0.9263418416005379]
Test set score: 0.9188395152405435


In [72]:
print(f"Best parameters: {best_KNN_params}")

Best parameters: {'n_neighbors': 16, 'p': 2, 'weights': 'distance'}


In [82]:
# joblib.dump(best_KNN_model, 'best_KNN_model.pkl')

['best_KNN_model.pkl']

In [73]:
### Logistic Regression
space = {
    'C': hp.loguniform('C', -4, 4),
    'penalty': hp.choice('penalty', ['l2'])
}

category_params = {
    'penalty': ['l2']
}

best_LR_model, best_LR_params = hyperopt_search(space, model=LR, algo=tpe.suggest, max_evals=100, categorical_params=category_params, 
                                                  x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

100%|██████████| 100/100 [00:38<00:00,  2.57trial/s, best loss: -0.9248720975993703]
Test set score: 0.9254498714652957


In [74]:
# print(best_LR_model.get_params())

{'C': 3.0231657252242194, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [75]:
print(f"Best parameters: {best_LR_params}")

Best parameters: {'C': 3.0231657252242194, 'penalty': 'l2'}


In [81]:
# joblib.dump(best_LR_model, 'best_LR_model.pkl')

['best_LR_model.pkl']

In [76]:
### LightGBM
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 150, 1),
    'max_depth': hp.quniform('max_depth', 5, 20, 1),
    'learning_rate': hp.loguniform('learning_rate', -4, 0),
    'num_leaves': hp.quniform('num_leaves', 10, 100, 1)
}

category_params = {}

int_params = ['n_estimators', 'max_depth', 'num_leaves']

best_LGBM_model, best_LGBM_params = hyperopt_search(space, model=lgbm.LGBMClassifier, algo=tpe.suggest, max_evals=100, categorical_params=category_params, integer_params=int_params, 
                                                  x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

100%|██████████| 100/100 [07:11<00:00,  4.32s/trial, best loss: -0.9291892913348629]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000301 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4080
[LightGBM] [Info] Number of data points in the train set: 10888, number of used features: 16
[LightGBM] [Info] Start training from score -2.345560
[LightGBM] [Info] Start training from score -3.257546
[LightGBM] [Info] Start training from score -2.126837
[LightGBM] [Info] Start training from score -1.346678
[LightGBM] [Info] Start training from score -1.926446
[LightGBM] [Info] Start training from score -1.917658
[LightGBM] [Info] Start training from score -1.640973
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Test set score: 0.9247153874403232


In [77]:
print(f"Best parameters: {best_LGBM_params}")

Best parameters: {'learning_rate': 0.04094892210733403, 'max_depth': 10, 'n_estimators': 127, 'num_leaves': 11}


In [79]:
# joblib.dump(best_LGBM_model, "best_LGBM_model.pkl")

['best_LGBM_model.pkl']

In [78]:
# ---
### Decision Tree
space = {
    'max_depth': hp.quniform('max_depth', 5, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 1)
}

category_params = {}

int_params = ['max_depth', 'min_samples_split', 'min_samples_leaf']

best_DT_model, best_DT_params = hyperopt_search(space, model=RFC, algo=tpe.suggest, max_evals=100, categorical_params=category_params, integer_params=int_params, 
                                                  x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

 15%|█▌        | 15/100 [01:03<06:00,  4.24s/trial, best loss: -0.9209225929483165]


KeyboardInterrupt: 

# -------------------------------------------

In [ ]:
----

In [ ]:
models = {
    "KNeighbors": KNC(),
    "LogisticRegression": LR(),
    "RandomForest": RFC(),
    "SVC": SVC(),
    # "GradientBoosting": GBC(),
    "LightGBM": lgbm.LGBMClassifier(verbose=-1)
}

def get_scores(models, xtrain, ytrain, xtest, ytest):
    for name, model in models.items():
        print(f"-----> {name}")
        model.fit(xtrain, ytrain)
        ypred = model.predict(xtest)
        model.score(xtest, ytest)
        print("---> Model score: ", model.score(xtest, ytest))
        reports = classification_report(ytest, ypred)
        print("---> Classification_reports: \n", reports)
        
        # scores = np.mean(cross_val_score(model, xtrain, ytrain, cv=5))
        # print("---> Cross-validation scores: \n", scores)
        print("============================================\n")

In [ ]:
get_scores(models, x_train, y_train, x_test, y_test)

# EDA

# Các thuộc tính
Area (A)	Diện tích:	Diện tích vùng của hạt đậu và số pixel nằm trong ranh giới của nó.

Perimeter (P)	Chu vi:	Chu vi của hạt đậu được định nghĩa là độ dài của đường viền.

Major axis length (L)	Độ dài trục chính:	Khoảng cách giữa hai đầu của đường thẳng dài nhất có thể vẽ trên hạt đậu.

Minor axis length (l)	Độ dài trục phụ:	Đường thẳng dài nhất có thể vẽ trên hạt đậu theo hướng vuông góc với trục chính.

Aspect ratio (K)	Tỷ lệ dài rộng:	Xác định mối quan hệ giữa L và l.

Eccentricity (Ec)	Độ lệch tâm:	Độ lệch tâm của hình elip có cùng các mô men như vùng hạt đậu.

Convex area (C)	Diện tích lồi:	Số pixel trong đa giác lồi nhỏ nhất có thể chứa diện tích của hạt đậu.

Equivalent diameter (Ed)	Đường kính tương đương:	Đường kính của một hình tròn có cùng diện tích với diện tích của hạt đậu.

Extent (Ex)	Độ bao phủ:	Tỷ lệ giữa số pixel trong khung bao và diện tích của hạt đậu.

Solidity (S)	Độ đặc:	Còn được gọi là độ lồi. Tỷ lệ giữa số pixel trong vỏ lồi và số pixel tìm thấy trong hạt đậu.

Roundness (R)	Độ tròn:	Được tính toán bằng công thức: (4piA)/(P^2)

Compactness (CO)	Độ gọn:	Đo độ tròn của một vật thể: Ed/L

ShapeFactor1 (SF1)	Hệ số hình dạng 1:	Tham số hình dạng bổ sung.

ShapeFactor2 (SF2)	Hệ số hình dạng 2:	Tham số hình dạng bổ sung.

ShapeFactor3 (SF3)	Hệ số hình dạng 3:	Tham số hình dạng bổ sung.

ShapeFactor4 (SF4)	Hệ số hình dạng 4:	Tham số hình dạng bổ sung.

Class (Seker, Barbunya, Bombay, Cali, Dermosan, Horoz and Sira)	Lớp	Thuộc tính này phân loại hạt đậu thành các loại khác nhau: Seker, Barbunya, Bombay, Cali, Dermosan, Horoz và Sira.


In [ ]:
dataset['Class'].value_counts().plot(kind='bar', xlabel='Types of bean', ylabel='Count')

### Drop outliner

In [ ]:
data_heads = x.columns
data_heads

In [ ]:
# # Biều đồ hist thêm đường trung vị
# fig, axes = plt.subplots(4,4, figsize = (24,24))
# for i, axe in enumerate(axes.flatten()):
#     sns.histplot(data = x[data_heads[i]], ax = axe)
#     median = x[data_heads[i]].median()
#     axe.set_title(f'{data_heads[i]} (Median = {median:0.1f})')
#     axe.axvline(median, color='red', lw=2, alpha=1)

In [ ]:
# # Biều đồ Boxplot để xem nhiễu
# fig, axes = plt.subplots(4, 4, figsize=(24,24))
# for i,axe in enumerate(axes.flatten()):
#     sns.boxplot(data = x[data_heads[i]], ax = axe)
#     axe.set_title(f'{data_heads[i]}')
# plt.tight_layout()

In [ ]:
# fig, axes = plt.subplots(8, 2, figsize=(24,48))
# for feature, ax in zip(x.columns, axes.flatten()):
#     sns.violinplot(data = dataset, x = 'Class', y = feature, ax = ax, inner="quartile" )
#     ax.set(xlabel = None)
#     ax.set_title(f'Phân bố {feature} từng loại')

In [ ]:
def drop_outliers(dataset, features):
    # Tính toán Z-score cho mỗi điểm dữ liệu
    z_scores = stats.zscore(dataset[features])

    # Xác định ngưỡng cho Z-score
    threshold = 3

    # Loại bỏ các outlier
    dataset_cleaned = dataset.loc[(z_scores < threshold).all(axis=1)]

    # In ra số lượng outlier đã loại bỏ
    outliers_removed = dataset.shape[0] - dataset_cleaned.shape[0]
    print("Số lượng outlier đã loại bỏ:", outliers_removed)
    return dataset_cleaned

In [ ]:
dataset_cleaned = drop_outliers(dataset, features)
x_train, x_test, y_train, y_test = prepare_data(dataset_cleaned, features, target, scaler, encoder)
# get_scores(models, x_train, y_train, x_test, y_test)

### Drop large corr

In [ ]:
# # correlation matrix
# plt.figure(figsize=(20,20))
# sns.heatmap(x.corr(),annot = True, cmap = 'viridis_r', fmt = '.2f');

In [ ]:
def drop_corr(dataset, features):
    # Đọc dữ liệu từ file CSV hoặc tạo ma trận tương quan từ dữ liệu hiện có
    corr_matrix = dataset[features].corr().abs()

    # Chọn ngưỡng tương quan
    threshold = 0.8

    # Tạo một bản sao của ma trận tương quan
    mask = corr_matrix.copy()

    # Thiết lập giá trị True cho các ô phía trên đường chéo chính
    # mask[np.triu_indices_from(mask.values)] = 0

    for i in range(len(mask.columns)):
        for j in range(i + 1):
            mask.iloc[i,j] = 0

    # print(mask)
    # Loại bỏ các biến dựa trên ngưỡng tương quan
    drop_cols = [column for column in mask.columns if any(mask[column] > threshold)]

    # Loại bỏ các cột từ DataFrame
    dataset_filtered = dataset.drop(drop_cols, axis=1)
    return dataset_filtered

In [ ]:
dataset_filtered = drop_corr(dataset_cleaned, features)

In [ ]:
x_train, x_test, y_train, y_test = prepare_data(dataset_filtered, dataset_filtered.columns[:-1], dataset_filtered.columns[-1], scaler, encoder)

In [ ]:
# get_scores(models, x_train, y_train, x_test, y_test)

In [ ]:
# Strongly_corr_features = dataset[["Area","Perimeter","AspectRation","Eccentricity","roundness","Compactness","Class"]]
# Strongly_corr_features.head()
# sns.set_theme(style="whitegrid")
# sns.pairplot(Strongly_corr_features, hue="Class")

In [ ]:
dataset.columns

In [ ]:
mi_scores = mutual_info_classif(x, y, random_state = 31)
mi_scores_df = pd.DataFrame({'Feature': x.columns, 'MI Score': mi_scores})
mi_scores_df = mi_scores_df.sort_values(by='MI Score', ascending=False).reset_index(drop=True)

# plt.figure(figsize=(20,10))
# sns.barplot(x='MI Score', y='Feature', data=mi_scores_df) 
# plt.xlabel('Mutual Information Score')
# # plt.ylabel('Feature')
# plt.title('Mutual Information Scores for Each Feature')
# # plt.gca().invert_yaxis() # sắp xếp lại theo giảm dần
# plt.show()

### Drop large corr and outliner

In [ ]:
data = drop_corr(dataset_cleaned, features)
x_train, x_test, y_train, y_test = prepare_data(data, data.columns[:-1], data.columns[-1], scaler, encoder)
# get_scores(models, x_train, y_train, x_test, y_test)

# * Data augmentation

## ** SMOTE

In [ ]:
! pip install imbalanced-learn


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
x_train, x_test, y_train, y_test = prepare_data(dataset, dataset.columns[:-1], dataset.columns[-1], scaler, encoder)

In [ ]:
x_train.shape

In [ ]:
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# for i in range(4):
#     x_train_resampled, y_train_resampled = smote.fit_resample(x_train_resampled, y_train_resampled)
print(x_train_resampled.shape)    
print(y_train_resampled.shape)

# get_scores(models, x_train_resampled, y_train_resampled, x_test, y_test)

In [ ]:
pd.DataFrame(y_train).value_counts()

## ** Feature perturbation

In [ ]:
x = dataset[features]
y = dataset[target]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=20)

In [ ]:
x_train.shape

In [ ]:
traindf = pd.DataFrame(x_train, columns=features)
traindf['Class'] = y_train

In [ ]:
countdf = traindf['Class'].value_counts()
for i in countdf.index:
    countdf[i] = 3500 / countdf[i]
countdf

In [ ]:
# Hàm để thêm nhiễu Gaussian

df1 = traindf.loc[traindf['Class'] == 'DERMASON']
df2 = traindf.loc[traindf['Class'] == 'SIRA']
df3 = traindf.loc[traindf['Class'] == 'SEKER']
df4 = traindf.loc[traindf['Class'] == 'HOROZ']
df5 = traindf.loc[traindf['Class'] == 'CALI']
df6 = traindf.loc[traindf['Class'] == 'BARBUNYA']
df7 = traindf.loc[traindf['Class'] == 'BOMBAY']



In [ ]:
# Hàm để thêm nhiễu Gaussian
def add_gaussian_noise(series, mean=0, std_dev=0.1):
    noise = np.random.normal(mean, std_dev, series.shape)
    return series + noise

def add_noise(df):
    for column in df.columns:
        if column != 'Class':
            df[column] = add_gaussian_noise(df[column])
    return df

In [ ]:
for i in countdf.index:
    if i == 'DERMASON':
        cp = df1.copy()
        for j in range(round(countdf[i]) - 2):
            df1 = pd.concat([df1, add_noise(cp)])
    elif i == 'SIRA':
        cp = df2.copy()
        for j in range(round(countdf[i]) - 2):
            df2 = pd.concat([df2, add_noise(cp)])
    elif i == 'SEKER':
        cp = df3.copy()
        for j in range(round(countdf[i]) - 2):
            df3 = pd.concat([df3, add_noise(cp)])
    elif i == 'HOROZ':
        cp = df4.copy()
        for j in range(round(countdf[i]) - 2):
            df4 = pd.concat([df4, add_noise(cp)])
    elif i == 'CALI':
        cp = df5.copy()
        for j in range(round(countdf[i]) - 2):
            df5 = pd.concat([df5, add_noise(cp)])
    elif i == 'BARBUNYA':
        cp = df6.copy()
        for j in range(round(countdf[i]) - 2):
            df6 = pd.concat([df6, add_noise(cp)])
    elif i == 'BOMBAY':
        cp = df7.copy()
        for j in range(round(countdf[i]) - 2):
            df7 = pd.concat([df7, add_noise(cp)])


In [ ]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7])

In [ ]:
df.shape

In [ ]:
df['Class'].value_counts()

In [ ]:
fp_data = pd.concat([traindf, df], axis=0)

In [ ]:
fp_data.shape

In [ ]:
fp_data['Class'].value_counts()

In [ ]:
# fp_x_train, fp_x_test, fp_y_train, fp_y_test = prepare_data(fp_data, fp_data.columns[:-1], fp_data.columns[-1], scaler, encoder)
x_fp_train = fp_data[features]
y_fp_train = fp_data[target]

In [ ]:
features

In [ ]:
target

In [ ]:
# get_scores(models, x_fp_train, y_fp_train, x_test, y_test)

## ** Synthetic data generation

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
dataset.head()

In [ ]:

# Chuyển đổi các đặc trưng phân loại sang số (encoding)
df_encoded = pd.get_dummies(dataset, columns=['Class'])

# Chuẩn bị dữ liệu cho mô hình GMM
X = df_encoded.values

# Khởi tạo và huấn luyện mô hình Gaussian Mixture Model
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(X)

# Sinh dữ liệu mới từ mô hình GMM
num_samples = 10000  # Số lượng mẫu mới cần tạo
X_synthetic = gmm.sample(num_samples)[0]

# Chuyển đổi dữ liệu sinh ngược lại thành DataFrame
df_synthetic = pd.DataFrame(X_synthetic, columns=df_encoded.columns)

# Đảo ngược quá trình encoding để có các đặc trưng phân loại
for class_label in dataset['Class'].unique():
    df_synthetic[class_label] = (dataset['Class'] == class_label).astype(int)

# Thêm nhãn giả lập (hoặc có thể sử dụng một mô hình để dự đoán nhãn)
df_synthetic['Label'] = np.random.choice(dataset['Class'].unique(), size=num_samples)

# In ra dữ liệu mới
print("Dữ liệu sau khi sinh từ GMM:")
df_synthetic.head()